# Data sources for MLlib

In [ ]:
# Reading CSVs

In [ ]:
df = spark.read.format("csv")\
    .option("inferSchema", True)\
    .option("header", True)\
    .load("../gramex/testlib/iris.csv")

In [ ]:
df.show(truncate=False)

In [ ]:
# From Pandas to Spark DataFrames

from sklearn.datasets import load_boston
import pandas as pd
boston = load_boston()

df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['price'] = boston.target

In [ ]:
sdf = sqlContext.createDataFrame(df)
sdf.show(truncate=False)

# Linear Regression with MLlib

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
lr = LinearRegression()
lr.fit(sdf)

## What went wrong?

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=[c for c in sdf.columns if c != 'price'],
                            outputCol='features')
dataset = assembler.transform(sdf)

In [ ]:
dataset.show()

In [ ]:
lr = LinearRegression(featuresCol='features', labelCol='price')
model = lr.fit(dataset)

In [ ]:
summary = model.evaluate(dataset)
print(summary.r2)

## Other regression methods

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor

In [ ]:
dtr = DecisionTreeRegressor(featuresCol='features', labelCol='price')
model = dtr.fit(dataset)

In [ ]:
summary = model.evaluate(dataset)
print(summary.r2)

### Different algorithms may have different interfaces!

In [ ]:
output = model.transform(dataset)
output.select('prediction').show()

In [ ]:
output.select('price').show()

## How do we evaluate this?

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol='price', predictionCol='prediction', metricName='r2')
evaluator.evaluate(output)

## Exercise: Use `RandomForestRegressor` (provided below) and report the R2 score

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

In [ ]:
# enter code here

# Classification with MLlib

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
df = spark.read.format("csv")\
    .option("inferSchema", True)\
    .option("header", True)\
    .load("../gramex/testlib/iris.csv")
df.printSchema()

In [ ]:
assembler = VectorAssembler(inputCols=[c for c in df.columns if c != 'species'],
                            outputCol='features')

In [ ]:
iris = assembler.transform(df)

In [ ]:
lr = LogisticRegression(featuresCol='features', labelCol='species')
model = lr.fit(iris)

## What went wrong?

In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='species', outputCol='speciesIndex')

iris = indexer.fit(df).transform(df)

iris.sample(fraction=0.1).show()

In [ ]:
assembler = VectorAssembler(inputCols=[c for c in df.columns if not c.startswith('species')],
                            outputCol='features')
dataset = assembler.transform(iris)

In [ ]:
dataset.printSchema()

In [ ]:
lr = LogisticRegression(featuresCol='features', labelCol='speciesIndex')
model = lr.fit(dataset)

In [ ]:
summary = model.evaluate(dataset)

In [ ]:
summary.accuracy

## That's too convenient

In [ ]:
trainData, testData = dataset.randomSplit([0.7, 0.3])

In [ ]:
model = lr.fit(trainData)
summary = model.evaluate(testData)

In [ ]:
summary.accuracy

## Exercise: Use the NaiveBayes classifier on the Iris dataset
### Check the accuracy for a 60-40 train / test split.
### Hint: Use the `MulticlassClassificationEvaluator`

In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
# enter code here